In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.2 MB/s eta 0:00:00


In [2]:
import openai
import numpy as np
import pandas as pd
import signal
import datetime
import time
import os

from google.cloud import bigquery
from typing import Dict, List, Union

In [3]:
API_KEY = "YOUR_API_KEY"
PROJECT_ID = "setalab"
DATASET_ID = "setalab.chatbot_finetune"

In [4]:
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ojSkZdGv3wxFeOKKlvRB5ZuOBJ6CaF&prompt=consent&access_type=offline&code_challenge=P7BNgzBVbEZns8rkAZOyDd1SYMpiEOaJEIkdSRB-nzA&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VPufVF1M15kBm4VUDxpwWlOduNdkrLa0vkmutiBeKOvUqJQpRdtyrwOn3N22TaaVw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to AD

In [24]:
# Set up OPENAI API
openai.api_key = API_KEY
# os.environ['OPENAI_API_KEY'] = API_KEY
# openai.api_key = os.getenv('OPENAI_API_KEY')

# Construct a Bigquery client object
client = bigquery.Client(project = PROJECT_ID)

# Define the model
#model = "gpt-3.5-turbo"

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [25]:
openai.api_key

# Preprocess the data

In [8]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


def list_tables(dataset_id: str) -> List[str]:

    # [START bigquery_list_tables]

    # TODO(developer): Set dataset_id to the ID of the dataset that contains
    #                  the tables you are listing.
    # dataset_id = 'your-project.your_dataset'

    tables_names = []
    tables = client.list_tables(dataset_id)  # Make an API request.

    print("Tables contained in '{}' ".format(dataset_id))
    for table in tables:
        tables_names.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))
    # [END bigquery_list_tables]
    return tables_names

list_tables(DATASET_ID)

Tables contained in 'setalab.chatbot_finetune' 


['setalab.chatbot_finetune.accounts',
 'setalab.chatbot_finetune.all_cell_locations',
 'setalab.chatbot_finetune.anomaly_detection_results',
 'setalab.chatbot_finetune.daily_aggregate',
 'setalab.chatbot_finetune.product']

In [10]:
def get_table_schema(table_name: Union[str, List[str]]) -> List[str]:
  #table_name = "setalab.chatbot_finetune.accounts"
  if isinstance(table_name, str):
    table_name = [table_name]  # Convert single table name to a list

  schema_list = []

  for table in table_name:
    table = client.get_table(table)
    schema = table.schema
    # Format the schema into the desired format
    formatted_schema = ', '.join(f'{field.name} {field.field_type}' for field in schema)
    schema_list.append(formatted_schema)

  table_schema = [
    {
      "name": table,
      "schema": schema_list
    } for table in table_name
  ]
  return table_schema

In [11]:
table_name = list_tables(DATASET_ID)
get_table_schema(table_name)


Tables contained in 'setalab.chatbot_finetune' 


[{'name': 'setalab.chatbot_finetune.accounts',
  'schema': ['ACCOUNT_ID INTEGER, ACCOUNT_TYPE INTEGER, NAME STRING, DIVISION STRING, WWW_ADDR STRING, SERVICE_NAME STRING, TMIN_MAXREQUEST INTEGER, TMIN_MAXTOTAL INTEGER, DATAINFO_TYPE INTEGER, DATAREPORT_TYPE INTEGER, MIN_MAXREQUEST INTEGER, PREDEVELOPER_ID INTEGER, NOT_USED INTEGER, MARKETING_TEAM STRING, REMARKS STRING, REGION_CODE INTEGER, PEAKTIME_BEGIN INTEGER, PEAKTIME_END INTEGER, REP_TIME INTEGER',
   'location_info STRING, mcc INTEGER, mnc INTEGER, lac INTEGER, cid INTEGER, longitude STRING, latitude STRING, source STRING, yearmonth_added STRING',
   'mean_absolute_error FLOAT, mean_squared_error FLOAT, mean_squared_log_error FLOAT, event_hour TIMESTAMP, account_id INTEGER, device_id INTEGER, server_ip STRING, server_ip_int STRING, domain STRING, server_port INTEGER, network_protocol STRING, network_protocol_int STRING, is_anomaly BOOLEAN',
   'event_date DATE, account_id INTEGER, server_ip STRING, product_name STRING, network_p

In [12]:
# Assuming table_name is a list of dictionaries
TABLE_NAME = [table["name"] for table in get_table_schema(table_name)]
SCHEMA = get_table_schema(table_name)[0]["schema"]
# print(TABLE_NAME)
# print(SCHEMA)

In [16]:
data = pd.read_csv("data_file.csv", skip_blank_lines = True, header = 0)

data['prompt'] = data['prompt'].str.strip()
data['completion'] = data['completion'].str.strip()
data['error'] = data['error'].str.strip()

In [17]:
data['prompt'] = data['prompt'] + "\n\n###\n\n"
data['completion'] = data['completion'] + " STOP"
data['error'] = data['error'] + "\n\n###\n\n"

In [18]:
data.head()

,prompt,completion,error
0,top 10 accounts with most transactions in all ...,"SELECT account_id, COUNT(*) AS num_transaction...",NaN
1,number of male and female customers\n\n###\n\n,"SELECT gender, COUNT(*) as count FROM setalab....",NaN
2,client's birth year distribution\n\n###\n\n,SELECT EXTRACT(YEAR FROM birth_date) AS birth_...,NaN
3,show top 5 accounts with the most devices in U...,"SELECT COUNT(*) AS count, account_id FROM all_...",NaN
4,show percentage of death over population by co...,"SELECT location, (total_deaths/population)*100...",NaN


In [19]:
data.to_json("finetune.jsonl",orient='records',lines=True)

In [20]:
import json
from sklearn.model_selection import train_test_split

json_file_name = "finetune.jsonl"

# Read the json file
data = []
with open(json_file_name, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Split the data into training and validation sets
_, val_data = train_test_split(data, test_size=1, random_state=42)

# Save the validation data to a new JSONL file
with open('finetune_prepared.jsonl', 'w') as file:
    for item in val_data:
        json.dump(item, file)
        file.write('\n')

# Fine Tune

In [21]:
train_file_name = "finetune.jsonl"
validation_file_name = "finetune_prepared.jsonl"

In [22]:
def check_status(training_id, validation_id):
    train_status = openai.File.retrieve(training_id)["status"]
    valid_status = openai.File.retrieve(validation_id)["status"]
    print(f'Status (training_file | validation_file): {train_status} | {valid_status}')
    return (train_status, valid_status)


In [28]:
from openai import cli

# Upload the training and validation dataset files to Azure OpenAI.

# Upload the data using the `openai.File.create` method
# response = openai.File.create(
#     file=open(train_file_name, 'rb'),
#     purpose='fine-tune'
# )
# training_id = response.id


# response = openai.File.create(
#     file=open(validation_file_name, 'rb'),
#     purpose='fine-tune'
# )
# validation_id = response.id


training_id = cli.FineTune._get_or_upload(train_file_name, True)
validation_id = cli.FineTune._get_or_upload(validation_file_name, True)

Found potentially duplicated files with name 'finetune.jsonl', purpose 'fine-tune' and size 2596 bytes
file-DFX4zt2iVwiA3dNm3B4DlcaW
file-nb7Lcud8HeyGhqMSpTIGrwGR
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 

Upload progress: 100%|██████████| 2.60k/2.60k [00:00<00:00, 1.92Mit/s]


Uploaded file from finetune.jsonl: file-4oF5KEtvRn0epPw8vpA5axvx
Found potentially duplicated files with name 'finetune_prepared.jsonl', purpose 'fine-tune' and size 271 bytes
file-4UwOtxrmJHrn8NGNtMVP2UzG
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 

Upload progress: 100%|██████████| 271/271 [00:00<00:00, 389kit/s]


Uploaded file from finetune_prepared.jsonl: file-TNE3iRgVZQQ0H5IZTEJLDF8d


In [29]:
print(f"Training File ID: {training_id}")
print(f"Validation File ID: {validation_id}")

Training File ID: file-4oF5KEtvRn0epPw8vpA5axvx
Validation File ID: file-TNE3iRgVZQQ0H5IZTEJLDF8d


In [30]:
check_status(training_id,validation_id)

Status (training_file | validation_file): processed | processed


('processed', 'processed')

# Create a customized model

In [227]:
create_args = {
	"training_file": training_id,
	"validation_file": validation_id,
	"model": "davinci",
	"n_epochs": 15,
	"batch_size": 3,
	"learning_rate_multiplier": 0.4
}

response = openai.FineTune.create(**create_args)
job_id = response["id"]
status = response["status"]

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ft-VMR5OJDE60QYIKHb2MojvT89.
Training Response: {
  "object": "fine-tune",
  "id": "ft-VMR5OJDE60QYIKHb2MojvT89",
  "hyperparams": {
    "n_epochs": 15,
    "batch_size": 3,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.4
  },
  "organization_id": "org-gag6d6xKTpJrl0TrHS85ygXB",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-4oF5KEtvRn0epPw8vpA5axvx",
      "purpose": "fine-tune",
      "filename": "finetune.jsonl",
      "bytes": 2596,
      "created_at": 1687399999,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-TNE3iRgVZQQ0H5IZTEJLDF8d",
      "purpose": "fine-tune",
      "filename": "finetune_prepared.jsonl",
      "bytes": 271,
      "created_at": 1687400001,
      "status": "processed",
      "status_details": null
    }
  ],
  "result_files": [],
  "created_at": 1687403935,
  "updated_at": 1

In [228]:

fine_tune_events = openai.FineTune.list_events(id=response.id)
fine_tune_events

<OpenAIObject list at 0x7f2d87e44590> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-VMR5OJDE60QYIKHb2MojvT89",
      "created_at": 1687403935
    }
  ]
}

In [229]:

retrieve_response = openai.FineTune.retrieve(id=response.id)
retrieve_response

<FineTune fine-tune id=ft-VMR5OJDE60QYIKHb2MojvT89 at 0x7f2d87e74720> JSON: {
  "object": "fine-tune",
  "id": "ft-VMR5OJDE60QYIKHb2MojvT89",
  "hyperparams": {
    "n_epochs": 15,
    "batch_size": 3,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.4
  },
  "organization_id": "org-gag6d6xKTpJrl0TrHS85ygXB",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-4oF5KEtvRn0epPw8vpA5axvx",
      "purpose": "fine-tune",
      "filename": "finetune.jsonl",
      "bytes": 2596,
      "created_at": 1687399999,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-TNE3iRgVZQQ0H5IZTEJLDF8d",
      "purpose": "fine-tune",
      "filename": "finetune_prepared.jsonl",
      "bytes": 271,
      "created_at": 1687400001,
      "status": "processed",
      "status_details": null
    }
  ],
  "result_files": [],
  "created_at": 1687403935,
  "updated_at": 1687

In [230]:
openai.FineTune.list()

<OpenAIObject list at 0x7f2d85cc21b0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-JazUeIZ5OWEEVh4YxDWqD4Wt",
      "hyperparams": {
        "n_epochs": 15,
        "batch_size": 3,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": 0.3
      },
      "organization_id": "org-gag6d6xKTpJrl0TrHS85ygXB",
      "model": "davinci",
      "training_files": [
        {
          "object": "file",
          "id": "file-8GWhcUzjEls7MhQpOQJ9pTG0",
          "purpose": "fine-tune",
          "filename": "training_data.jsonl",
          "bytes": 201,
          "created_at": 1686626333,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [
        {
          "object": "file",
          "id": "file-Rr9pN5khLXXnJvTWDRO0Uxut",
          "purpose": "fine-tune",
          "filename": "validation_data.jsonl",
          "bytes": 201,
          "created_at": 1686626335,
          "

In [239]:
def signal_handler(sig, frame):
	status = openai.FineTune.retrieve(job_id).status
	print(f"Stream interrupted. Job is still {status}.")
	return

print(f'Streaming events for the fine-tuning job: {job_id}')
signal.signal(signal.SIGINT, signal_handler)
events = openai.FineTune.stream_events(job_id)

try:
	for event in events:
		print(f'{datetime.datetime.fromtimestamp(event["created_at"])} {event["message"]}')

except Exception:
	print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ft-VMR5OJDE60QYIKHb2MojvT89
2023-06-22 03:18:55 Created fine-tune: ft-VMR5OJDE60QYIKHb2MojvT89
2023-06-22 03:19:41 Fine-tune costs $0.24
2023-06-22 03:19:41 Fine-tune enqueued. Queue number: 0
2023-06-22 03:19:45 Fine-tune started
2023-06-22 03:24:12 Completed epoch 1/15
2023-06-22 03:24:15 Completed epoch 2/15
2023-06-22 03:24:18 Completed epoch 3/15
2023-06-22 03:24:21 Completed epoch 4/15
2023-06-22 03:24:24 Completed epoch 5/15
2023-06-22 03:24:28 Completed epoch 6/15
2023-06-22 03:24:31 Completed epoch 7/15
2023-06-22 03:24:34 Completed epoch 8/15
2023-06-22 03:24:38 Completed epoch 9/15
2023-06-22 03:24:41 Completed epoch 10/15
2023-06-22 03:24:44 Completed epoch 11/15
2023-06-22 03:24:47 Completed epoch 12/15
2023-06-22 03:24:50 Completed epoch 13/15
2023-06-22 03:24:53 Completed epoch 14/15
2023-06-22 03:24:57 Completed epoch 15/15
2023-06-22 03:25:43 Uploaded model: davinci:ft-personal-2023-06-22-03-25-42
2023-06-22 03:25:44 Uploaded r

In [240]:
# Get the status of our fine-tune job.
status = openai.FineTune.retrieve(id=job_id)["status"]

# If the job isn't yet done, poll it every 2 seconds.
if status not in ["succeeded", "failed"]:
    print(f'Job not in terminal status: {status}. Waiting.')
    while status not in ["succeeded", "failed"]:
        #time.sleep(2)
        status = openai.FineTune.retrieve(id=job_id)["status"]
        print(f'Status: {status}')
else:
    print(f'Fine-tune job {job_id} finished with status: {status}')

# Check if there are other fine-tune jobs in the subscription.
# Your fine-tune job may be queued, so this is helpful information to have
# if your fine-tune job hasn't yet started.
print('Checking other fine-tune jobs in the subscription.')
result = openai.FineTune.list()
print(f'Found {len(result.data)} fine-tune jobs.')

Fine-tune job ft-VMR5OJDE60QYIKHb2MojvT89 finished with status: succeeded
Checking other fine-tune jobs in the subscription.
Found 47 fine-tune jobs.


In [241]:
# Retrieve the name of the customized model from the fine-tune job.
result = openai.FineTune.retrieve(id=job_id)
if result["status"] == 'succeeded':
    model = result["fine_tuned_model"]

# Create the deployment for the customized model, using the standard scale type without specifying a scale
# capacity.
print(f'Creating a new deployment with model {model}')
#result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard", "capacity": None})
# Retrieve the deployment job ID from the results.
deployment_id = result["id"]

Creating a new deployment with model davinci:ft-personal-2023-06-22-03-25-42


In [242]:
model

'davinci:ft-personal-2023-06-22-03-25-42'

# Helper function

In [248]:
def generate_sql_query(question, temperature: float = 0.7):
    print(f"Question:\n{question}")
    prompt_without_context = question
    prompt_with_context = f"Given the table '{table_name}', please write an SQL query to {question}"

    messages = [
        {"role": "system", "content": f"Use the reference table name: {TABLE_NAME}, schema : {SCHEMA}"},
        {"role": "system", "content": "Based on the user's question, compose a SQL query in Google BigQuery dialect to retrieve the desired information from the table."},
        {"role": "user", "content": prompt_without_context},
        # {"role": "assistant", "content": prompt_with_context},
        # Guidelines to improve query results
        {"role": "assistant", "content": "If the user is querying by a specific column, consider including a GROUP BY X clause for that column in the SQL query."},
        # {"role": "assistant", "content": "When using the AS clause to rename columns, avoid including white spaces and quotation marks in the new column names."},
        {"role": "assistant", "content": "When using aggregate functions, ensure that all non-aggregated columns are included in the GROUP BY X clause."},
        {"role": "assistant", "content": "If the user does not specify a time period, the query should consider all dates available in the table."},
        {"role": "assistant", "content": "If the user does not provide specific requirements for numeric values, consider using the average value as a default."},
        {"role": "system", "content": "Please provide the SQL query only. Do not provide any explanations."},
    ]

    # response = openai.ChatCompletion.create(
    #     model=model,
    #     messages=messages,
    #     max_tokens=100,
    #     temperature=temperature,
    #     n=1,
    #     stop=[" STOP"], # stop=None
    # )

    response = openai.Completion.create(
      model=model,
      #messages = messages,
      prompt=prompt,
      temperature=temperature,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=[ " STOP"]
    )

    job_id = response["id"]
    print(f'Fine-tunning model with jobID: {job_id}.')

    sql_query = response['choices'][0]['text'].replace('\n',' ').replace(' .','.').strip()
    #sql_query = sql_query.strip().strip("```")
    return sql_query



In [249]:
prompt = "show client's total spending by birth year\n\n###\n\n"
generate_sql_query(prompt)

Question:
show client's total spending by birth year

###


Fine-tunning model with jobID: cmpl-7U5GMuuxTBGB6qFS6qzpaEHeWn8eu.


'SELECT birth_year, COUNT(*) AS num_spendings FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year'

In [250]:
from google.api_core.exceptions import BadRequest

def execute_sql_query(sql_query: str) -> pd.DataFrame:
  """Run a SQL queries on predefined project.
    Note that the `client` variable for BigQuery client must be initialized before running this method

    Args:
      sql_query (str): Query to run

    Returns:
      pd.DataFrame: Pandas dataframe containing the results of the query
  """

  try:
        result = client.query(sql_query).to_dataframe()
        return result
  except BadRequest as e:
        error_position = e.errors[0]['location']
        error_message = e.errors[0]['message']
        print(f"BadRequest Error: {error_message} at {error_position}")
  # result = client.query(sql_query).to_dataframe()
  # return result

In [252]:
def ask_question_and_validate(question: str = None, temperature: float=0.7) -> pd.DataFrame:
    user_input = input("Input question:") if not question else question
    user_input += "\n\n###\n\n"
    sql_query = generate_sql_query(user_input, temperature)
    print("\nGenerated SQL Query:\n", sql_query)

    # Execute the generated SQL query
    result = execute_sql_query(sql_query)
    # print("Result:", result)
    return result

In [253]:
request = "show 10 accounts has time stamp"
ans = ask_question_and_validate(request)
ans

Question:
show 10 accounts has time stamp

###


Fine-tunning model with jobID: cmpl-7U5HHhPoRx2drsleVOj6bpaIWeYSc.

Generated SQL Query:
 SELECT year(birth_date) AS birth_year, SUM(spending) AS total_spending FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Function not found: year at [1:8] at query


In [254]:
ans = ask_question_and_validate(
    "client's birth year distribution"
)
ans

Question:
client's birth year distribution

###


Fine-tunning model with jobID: cmpl-7U5Hc47dAziMZKnPdVqUxjPMEuAQ9.

Generated SQL Query:
 SELECT birth_year, SUM(spending) AS total_spending FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Unrecognized name: birth_year; Did you mean birth_date? at [1:8] at query


In [255]:
ans = ask_question_and_validate(
    "show average percentage of Infection Rate compared to Population by country, return top 10"
)
ans

Question:
show average percentage of Infection Rate compared to Population by country, return top 10

###


Fine-tunning model with jobID: cmpl-7U5HlhBEiAh37hm0VQ36hX55DHsvk.

Generated SQL Query:
 SELECT birth_year, COUNT(*) AS num_clients FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Unrecognized name: birth_year; Did you mean birth_date? at [1:8] at query


In [256]:
ans = ask_question_and_validate(
    "show top 3 contintents with the highest average death count per population"
)
ans

Question:
show top 3 contintents with the highest average death count per population

###


Fine-tunning model with jobID: cmpl-7U5HpoZkqm67CB3XXcWYPHPTmpNEr.

Generated SQL Query:
 SELECT birth_year, COUNT(*) AS num_clients FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Unrecognized name: birth_year; Did you mean birth_date? at [1:8] at query


In [257]:
ans = ask_question_and_validate(
    "show the bank receiving the most money for each customer. Specify where you get the column client_id in every subquery if necessary. Table account does not have the column client_id"
)
ans

Question:
show the bank receiving the most money for each customer. Specify where you get the column client_id in every subquery if necessary. Table account does not have the column client_id

###


Fine-tunning model with jobID: cmpl-7U5HvDp7AinLraE0kpDK18j2s3xF9.

Generated SQL Query:
 SELECT year(birth_date) AS birth_year, SUM(spending) AS total_spending FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Function not found: year at [1:8] at query


In [258]:
ans = ask_question_and_validate(
    "show the fastest rep time and '@gmail.com' in name, return top 10"
)
ans

Question:
show the fastest rep time and '@gmail.com' in name, return top 10

###


Fine-tunning model with jobID: cmpl-7U5I0vUNyGU3pufCy9BRr6X9hO1M0.

Generated SQL Query:
 SELECT year(birthdate) AS birth_year, COUNT(*) AS num_clients FROM setalab.financial.client GROUP BY birth_year ORDER BY birth_year
BadRequest Error: Function not found: year at [1:8] at query
